# Capstone Project: Amazon Review Classification (Part 4)
Author: **Steven Lee**

# Transformers and Pretrained Models

In [1]:
import pandas as pd
from random import sample

# Set pandas display options.
pd.set_option('display.max_colwidth', None)

from transformers import pipeline

import warnings
# warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [42]:
# Read in LDA data.
reviews = pd.read_csv("../data/reviews_demo.csv")
total_rev = reviews.shape[0]

In [147]:
# Initialise pipeline transformers.
sa_pipeline = pipeline('sentiment-analysis')
bert_cl_pl = pipeline("zero-shot-classification", model="bert-base-cased")
fb_cl_pl = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ['tool', 'cord', 'floor', 'knife', 'light', 'love', 'glove', 'heavy', 'color', 'used', 'much', 'clamp', 
                    'worked', 'bulb', 'work', 'wire', 'well', 'blade', 'water', 'door', 'would', 'device', 'quality', 'doe', 
                    'plastic', 'easy', 'great', 'mouth', 'kitchen']

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

## Sentiment Analysis

In [134]:
# Randomly select a review.
rev_idx = sample(range(total_rev), 1)
topic, rev_txt, rev_doc, rating = reviews.loc[rev_idx[0], ['dmnt_topic', 'reviewText', 'document', 'overall']]

In [135]:
print(f"Review Text: {rev_txt}")
output = sa_pipeline(rev_txt)
output

Review Text: great tools always pick these . THE BEST


[{'label': 'POSITIVE', 'score': 0.9998624324798584}]

## Zero-Shot Classification

In [136]:
print(f"Topic Candidate Labels: {candidate_labels}")
results_bert = bert_cl_pl(rev_doc, candidate_labels)
results_fb = fb_cl_pl(rev_doc, candidate_labels)

Topic Candidate Labels: ['tool', 'cord', 'floor', 'knife', 'light', 'love', 'glove', 'heavy', 'color', 'used', 'much', 'clamp', 'worked', 'bulb', 'work', 'wire', 'well', 'blade', 'water', 'door', 'would', 'device', 'quality', 'doe', 'plastic', 'easy', 'great', 'mouth', 'kitchen']


In [137]:
def print_results(results):
    labels = results['labels']
    scores = results['scores']
    output = []

    for i in range(10):
        pair = ()
        pair = (labels[i], round(scores[i], 4))
        output.append(pair)
        
    return output

output = print_results(results_bert)
print(f"Bert Model Label-Score Pairs: {output}")
output = print_results(results_fb)
print(f"\nBart Model Label-Score Pairs: {output}")

Bert Model Label-Score Pairs: [('tool', 0.0569), ('knife', 0.0459), ('blade', 0.0448), ('door', 0.0358), ('great', 0.0357), ('would', 0.0352), ('cord', 0.0347), ('device', 0.0346), ('clamp', 0.0346), ('worked', 0.0346)]

Bart Model Label-Score Pairs: [('tool', 0.2537), ('great', 0.1504), ('well', 0.1196), ('quality', 0.0906), ('device', 0.0827), ('worked', 0.0682), ('used', 0.0539), ('much', 0.0463), ('would', 0.0397), ('doe', 0.0095)]
